In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [3]:
max_size = 57
max_length = 56

In [4]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [5]:
import libs_qrem
importlib.reload(libs_qrem)

<module 'libs_qrem' from '/Users/yangbo/opt/anaconda3/envs/qiskitenv/lib/python3.9/site-packages/libs_qrem.cpython-39-darwin.so'>

In [6]:
xz_ignis_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    xz_ignis_mitigator_list.append(meas_fitter.filter)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_ignis_mitigator_list: ", len(xz_ignis_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of xz_ignis_mitigator_list:  56


In [7]:
zx_ignis_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    zx_ignis_mitigator_list.append(meas_fitter.filter)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_ignis_mitigator_list: ", len(zx_ignis_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of zx_ignis_mitigator_list:  56


In [8]:
ignis_hist_list = []
xz_ignis_total_times = []
zx_ignis_total_times = []
for i in range(9):
    print(i + 2, "qubits")
    
    t1 = time.perf_counter()
    ignis_hist_list.append( xz_ignis_mitigator_list[i].apply(raw_hist_list[2 * i]) )
    t2 = time.perf_counter()
    xz_ignis_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    ignis_hist_list.append( zx_ignis_mitigator_list[i].apply(raw_hist_list[2 * i + 1]) )
    t4 = time.perf_counter()
    zx_ignis_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 0.008804213000001226 s
zx 0.006805534999999807 s

3 qubits
xz 0.020689488000000367 s
zx 0.023538367999998755 s

4 qubits
xz 0.08934289199999945 s
zx 0.0849118390000001 s

5 qubits
xz 0.3463105589999991 s
zx 0.3493858670000005 s

6 qubits
xz 1.6843746680000002 s
zx 1.7289630030000005 s

7 qubits
xz 7.192805320999998 s
zx 7.404717606999995 s

8 qubits
xz 32.933201098 s
zx 32.840831463 s

9 qubits
xz 137.834563271 s
zx 137.334000232 s

10 qubits
xz 647.125990951 s
zx 645.6466338820001 s



In [9]:
with open("./pkls/ignis_hist_list.pkl", "wb") as f:
    pickle.dump(ignis_hist_list, f)

In [10]:
with open("./pkls/ignis_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_ignis_total_times, "zx": zx_ignis_total_times}, f)